## Basic Lore Builder - Using Chroma DB & Guidance to Generate Worlds



This is a tutorial that explores basic world-building through retrieval augmented generation (RAG). This approach combines contrained language models (via guidance) with vector storage (via ChromaDB) to create and manage fictional worlds; the system uses semantic search and stateless generation to build coherent, interconnected lore.

Note, this is based on the [Loremaster 6000 project](https://github.com/dottxt-ai/demos/lore-generator).


### Library Imports

First we need to import all the necessary libraries.

In [1]:
# Standard library imports
import os
import math
from typing import List, Optional, Any, Dict
from enum import Enum

# Third-party imports
import torch
from pydantic import BaseModel, Field
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import chromadb


# Guidance imports
import guidance
from guidance import models, gen, select, user, system, assistant
from guidance.chat import ChatTemplate

### Data Models


The data models provide the foundational structure for our world-building system. These models
define both the schema for our data and the validation rules that ensure consistency.

Key concepts:
1. Settings types are enforced through Enums
2. Data validation through Pydantic
3. Clean serialization for database storage
4. Type hints for better IDE support

In [2]:
#######################################
# Data Models
#######################################

class SettingType(str, Enum):
    """Defines the possible world settings/genres.
    
    Using str as base class enables automatic serialization."""
    science_fiction = "science_fiction"
    fantasy = "fantasy" 
    horror = "horror"
    cyberpunk = "cyberpunk"
    steampunk = "steampunk"
    post_apocalyptic = "post_apocalyptic"
    magical_realism = "magical_realism"

class World(BaseModel):
    """Defines the structure of a world.
    
    Attributes:
        setting: The genre/type of the world
        world_description: Detailed world description"""
    setting: SettingType
    world_description: str

class LoreEntry(BaseModel):
    """Defines a piece of lore.
    
    Attributes:
        name: Title/identifier
        content: Main lore text
        keywords: Search/categorization terms"""
    name: str
    content: str  
    keywords: List[str]

### Embeddings

The embeddings system handles semantic search capabilities:

```python
class SentenceTransformerEmbeddings(BaseModel):
    model_name: str = Field(default="sentence-transformers/all-mpnet-base-v2")
    # ...
```

Key concepts:
- Uses sentence-transformers to convert text to vectors
- Handles both single and batch embedding
- Supports multi-processing for large-scale operations



In [3]:
#######################################
# Embeddings
#######################################

class SentenceTransformerEmbeddings(BaseModel):
    """Manages document embeddings for semantic search."""
    
    model_name: str = Field(default="sentence-transformers/all-mpnet-base-v2")
    cache_folder: Optional[str] = None
    model_kwargs: Dict[str, Any] = Field(default_factory=dict)
    encode_kwargs: Dict[str, Any] = Field(default_factory=dict)
    query_encode_kwargs: Dict[str, Any] = Field(default_factory=dict)
    multi_process: bool = False
    show_progress: bool = False
    
    def __init__(self, **kwargs):
        """Initialize embedding model."""
        super().__init__(**kwargs)
        try:
            from sentence_transformers import SentenceTransformer
        except ImportError:
            raise ImportError("Please install sentence-transformers")
            
        self._client = SentenceTransformer(
            self.model_name,
            cache_folder=self.cache_folder,
            **self.model_kwargs
        )
    
    def _embed(self, texts: List[str], encode_kwargs: Dict[str, Any]) -> List[List[float]]:
        """Compute embeddings for texts."""
        texts = [text.replace("\n", " ") for text in texts]
        
        if self.multi_process:
            pool = self._client.start_multi_process_pool()
            embeddings = self._client.encode_multi_process(texts, pool)
            self._client.stop_multi_process_pool(pool)
        else:
            embeddings = self._client.encode(
                texts,
                show_progress_bar=self.show_progress,
                **encode_kwargs
            )
        
        return embeddings.tolist()
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed multiple documents."""
        return self._embed(texts, self.encode_kwargs)
    
    def embed_query(self, text: str) -> List[float]:
        """Embed a single query."""
        embed_kwargs = self.query_encode_kwargs or self.encode_kwargs
        return self._embed([text], embed_kwargs)[0]

### Chat Template 

In [4]:
#######################################
# Chat Template 
#######################################

class InstructTemplate(ChatTemplate):
    """Formats chat messages for instruction-tuned models."""
    
    def get_role_start(self, role_name: str) -> str:
        """Get the prefix for a role's message."""
        prefixes = {
            "system": "System: ",
            "user": "User: ",
            "assistant": "Assistant: "
        }
        if role_name not in prefixes:
            raise ValueError(f"Unsupported role: {role_name}")
        return prefixes[role_name]

    def get_role_end(self, role_name: Optional[str] = None) -> str:
        """Get the suffix for a role's message."""
        return "\n"

### Lore Manager

The LoreManager class orchestrates the entire system:

```python
class LoreManager:
    def __init__(self, model_path: str, embeddings_path: str):
        self.vectorstore = None
        self.embeddings = None 
        self.model = None
```

Key responsibilities:
1. Initialize and manage the language model
2. Handle the vector store connection
3. Coordinate lore generation and querying
4. Manage resource cleanup



In [5]:
#######################################
# Lore Manager
#######################################

class LoreManager:
    """Central manager for the lore system."""
    
    def __init__(self, model_path: str, embeddings_path: str):
        """Initialize all components.
        
        Args:
            model_path: Path to language model
            embeddings_path: Path to embedding model"""
        self.vectorstore = None
        self.embeddings = None 
        self.model = None
        self.setup(model_path, embeddings_path)
    
    def setup(self, model_path: str, embeddings_path: str):
        """Set up models and vector store."""
        
        # Initialize embeddings
        self.embeddings = SentenceTransformerEmbeddings(
            model_name=embeddings_path,
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True}
        )

        self.client = chromadb.PersistentClient(path="./lore.db")
        self.vectorstore = self.client.get_or_create_collection(name="lore_collection")

        
        # Initialize language model
        self.model = self._setup_model(model_path)
    
    def _setup_model(self, model_path: str) -> models.Transformers:
        """Initialize the language model."""
        
        # Configure quantization
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,  
            bnb_4bit_compute_dtype=torch.float16
        )
        
        # Load model and tokenizer
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="cuda",
            quantization_config=quant_config,
            torch_dtype=torch.float16
        )
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        
        # Create guidance model
        return models.Transformers(
            model=model,
            tokenizer=tokenizer,
            chat_template=InstructTemplate,
            device_map="cuda"
        )
        
    def add_lore(self, entry: LoreEntry):
        """Add a lore entry to the vector store."""
        self.vectorstore.add(
            ids=[entry.name],
            documents=[entry.content],
            metadatas=[{
                "keywords": " ".join(entry.keywords)
            }]
        )

    
    def cleanup(self):
        """Clean up resources without calling persist explicitly."""
        if hasattr(self.model, 'cleanup'):
            self.model.cleanup()
            
        self.vectorstore = None
        self.embeddings = None
        self.model = None
        self.client = None



### Generation Functions

The world-building system relies on two core guidance functions that work together to generate and query lore. The first function, `generate_lore`, creates new pieces of lore while maintaining consistency with the existing world. It takes a World object that defines the setting and description, a focus topic to guide the generation, and access to the vector store of existing lore. When called, it retrieves relevant existing lore entries and uses them as context to generate new, thematically consistent content.

The second function, `query_lore`, helps maintain consistency by enabling semantic searches through the existing lore. When you need to check if new content aligns with established facts, this function searches the vector store for relevant entries and synthesizes an answer based on the retrieved information. Both functions are decorated with `@guidance(stateless=True)` to ensure they generate consistent outputs without maintaining internal state between calls.

These functions work together in a cycle - `generate_lore` creates new content while consulting existing lore through `query_lore` to maintain consistency. The stateless nature of these functions means they can be called independently and still produce coherent results, making the system more reliable and easier to test. For example, when generating lore about a magical artifact, `generate_lore` might first use `query_lore` to check existing information about magic systems in the world before creating new content that fits those established rules.


In [6]:
#######################################
# Generation Functions
#######################################

@guidance(stateless=True)
def generate_lore(lm, world: World, focus: str, vectorstore):
    """Generate focused lore entry.
    
    Args:
        lm: Language model
        world: World context
        focus: Topic to focus on
        vectorstore: Vector store instance"""
    
    # Get relevant existing lore
    results = vectorstore.query(
        query_texts=[focus],
        n_results=3
    )
    context = "\n- ".join(results["documents"][0]) if results["documents"] else "No existing lore."
    context = f"- {context}" if results["documents"] else context

    with system():
        lm += f"""You are a world-building assistant that generates detailed, consistent lore entries."""

    with user():
        lm += f"""### Instruction
Create a new lore entry for a fictional world that expands the existing world-building in a consistent way.

### Input
World Setting: {world.setting.value}
World Description: {world.world_description}
Focus Topic: {focus}

Existing Relevant Lore:
{context}

Requirements:
1. Create content focused on {focus}
2. Ensure it fits the {world.setting.value} theme
3. Build upon and stay consistent with existing lore
4. Include rich, specific details
5. Avoid contradicting established facts

Output Format:
Provide the response in this exact format:
Name: [A clear, descriptive title]
Content: [The main lore text]
Keywords: [3-5 relevant search terms]

### Output
"""

    with assistant():
        lm += "Name: " + gen(name="name", stop="\n", max_tokens=150) + "\n"
        lm += "Content: " + gen(name="content", stop="\n", max_tokens=350) + "\n"
        lm += "Keywords: " + gen(name="keywords", stop="\n", max_tokens=50)
    
    return lm

@guidance(stateless=True)
def query_lore(lm, query: str, vectorstore):
    """Answer a lore query.
    
    Args:
        lm: Language model
        query: Search query
        vectorstore: Vector store instance"""
    
    # Get relevant lore
    results = vectorstore.query(query_texts=query, n_results=3)
    context = "\n- ".join(results["documents"][0]) if results["documents"] else "No existing lore."
    context = f"- {context}" if results["documents"] else context

    with system():
        lm += """You are a knowledgeable lore expert who answers questions based strictly on provided information."""

    with user():
        lm += f"""### Instruction
Answer the query by analyzing and synthesizing information from the provided lore. Only use facts explicitly stated in the lore.

### Input
Query: {query}

Available Lore:
{context}

Requirements:
1. Answer should be detailed and specific
2. Only use information from the provided lore
3. Clearly indicate if information is uncertain or missing
4. Provide a single comprehensive paragraph

### Output
"""

    with assistant():
        lm += gen(name="response", stop="\n\n", max_tokens=500, temperature=0.8)
    
    return lm

### Main Usage Example

The `build_focused_world` function serves as a comprehensive world-building orchestrator, taking a world type (like fantasy or horror) and a core concept as inputs. At its heart, it uses the LoreManager to coordinate between a language model (Phi-3.5) and a vector database, enabling the generation and storage of interconnected world elements.

The function works in two main phases. First, it generates foundational aspects of the world - from basic principles to historical events - storing each piece of generated lore in the vector database. This builds up a knowledge base about the world. Second, it analyzes the relationships between these elements by querying the stored lore, asking questions about how different aspects interact and influence each other. For example, it might explore how historical events have shaped the current state of the world or how different factions interact with important locations.

What makes this approach powerful is its use of the vector store to maintain context between generations. Each new piece of lore can build upon and reference previously generated content, creating a coherent and interconnected world. The function wraps all this functionality in a try-finally block to ensure proper resource cleanup, making it safe for production use. The result is a systematic approach to world-building that combines the creative power of language models with the structured storage and retrieval capabilities of vector databases.

In [7]:
def build_focused_world(world_type: SettingType, core_concept: str):
    """Build a world focused on a specific concept."""
    
    #MODEL_PATH = r'C:\Users\polym\.cache\huggingface\hub\models--microsoft--Phi-3.5-mini-instruct\snapshots\af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0'
    
    MODEL_PATH = r"C:\Users\polym\.cache\huggingface\hub\models--tiiuae--Falcon3-7B-Instruct\snapshots\28519b87831eaf6dbe6962f889b82b5a25b5d940"
    
    EMBEDDINGS_PATH = r"C:\Users\polym\.cache\torch\sentence_transformers\BAAI_bge-large-en-v1.5"
    
    lore_manager = LoreManager(MODEL_PATH, EMBEDDINGS_PATH)
    
    try:
        world = World(
            setting=world_type,
            world_description=core_concept
        )
        
        # Generate foundation
        print("\nGenerating foundation...")
        foundation_aspects = [
            "basic principles",
            "key powers",
            "major factions",
            "important locations",
            "historical events"
        ]
        
        for aspect in foundation_aspects:
            result = lore_manager.model + generate_lore(
                world,
                aspect,
                lore_manager.vectorstore
            )
            
            if result:
                lore_manager.add_lore(LoreEntry(
                    name=result["name"],
                    content=result["content"],
                    keywords=[k.strip() for k in result["keywords"].split(",")]
                ))
        
        # Query relationships
        print("\nAnalyzing connections...")
        relationship_queries = [
            "How do the basic principles affect the key powers?",
            "What role do major factions play in important locations?",
            "How have historical events shaped the current state?",
            "What conflicts exist between different factions?",
            "How do locations influence power dynamics?"
        ]

        qa = []
        
        for query in relationship_queries:
            response = lore_manager.model + query_lore(
                query,
                lore_manager.vectorstore
            )
            qa.append({"question": query, "answer":response['response']})

        for x in qa:
            print(x)
        
    finally:
        lore_manager.cleanup()

In [8]:
build_focused_world(world_type=SettingType.horror, core_concept= "A summer camp in Texas during the summer of 1969.")

{'question': 'How do the basic principles affect the key powers?', 'answer': "The basic principles, as outlined in the lore, profoundly affect the key powers of Camp Nightshade's factions, The Order of the Arcane and the Whisperers' Guild. For The Order, these principles are the arcane rituals and incantations that govern their interaction with the spirits bound to the land. The Midnight Summoning, a ritual steeped in ancient lore, is the cornerstone of their power, allowing them to harness the spirits' abilities, such as elemental control and psychic manipulation. These powers are a direct result of their adherence to the basic principles, which emphasize control and dominance over the spirits. In contrast, the Whisperers' Guild operates under a fundamentally different set surfers, prioritizing empathy and understanding over dominance. Their basic principles involve a more personal and intimate relationship with the spirits, fostering alliances through communication and understanding 

### Current Limitations


1. Memory Management
- Relies on garbage collection
- No explicit memory limits for vector store
- Potential for memory leaks in long sessions

2. Context Window
- Limited use of context window
- No intelligent chunking of long texts
- Could benefit from better context management

3. Consistency
- No explicit consistency checking between entries
- Could generate contradicting lore
- No temporal tracking

4. Structure
- Flat structure of lore entries
- No hierarchical relationships
- Limited metadata capabilities



### Ideas

- Knowledge Graph RAG
    - Allows for a more structured and atomic RAG with clearly defined relationships.
    - A schema might be useful to clearly define everything.
    - Properties could contain additional information useful for games, such as statistics.
- Multilayer graphs
    - To provide top level information, such as linking temporal relationships or logical arguments to a lower level knowledge graph.
    - Rules and a logic might enable something like PyReason.
    - Bottom layer could be game agnostic entities and their relationships, while higher levels could define game specific rules.
- Knowledge graph game engine
    - Objective world graph vs subjective player graphs
    - Action availability based on pattern matching between objective and subjective graphs

    

## Advanced Lore Builder 

In [1]:
# Standard library imports
from enum import Enum
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field

class SettingType(str, Enum):
    """Defines the possible world settings/genres.
    Using str as base class enables automatic JSON serialization."""
    science_fiction = "science_fiction"
    fantasy = "fantasy" 
    horror = "horror"
    cyberpunk = "cyberpunk"
    steampunk = "steampunk"
    post_apocalyptic = "post_apocalyptic"
    magical_realism = "magical_realism"

class World(BaseModel):
    """Defines the structure and rules of a world.
    
    Attributes:
        setting: The genre/type of the world
        world_description: Detailed world description
    """
    setting: SettingType
    world_description: str

    def to_context(self) -> str:
        """Converts world info to a prompt-friendly format"""
        return f"Genre: {self.setting}\nWorld Description: {self.world_description}"

class LoreEntry(BaseModel):
    """A piece of lore that adds detail to the world.
    
    Attributes:
        name: Title/identifier for the lore piece
        content: Main lore text
        keywords: Search/categorization terms
    """
    name: str
    content: str
    keywords: List[str]

class LoreProposal(BaseModel):
    """A proposed piece of lore before refinement.
    
    Attributes:
        initial_proposal: The first draft of the lore
        questions: Questions to refine the lore
        reasoning: Reasoning steps taken
    """
    initial_proposal: str
    questions: List[str]
    reasoning: List[str]

class LoreRefinement(BaseModel):
    """Response to refining a lore proposal.
    
    Attributes:
        reasoning: Steps taken to refine
        answer: The refined answer
    """
    reasoning: List[str]
    answer: str

In [2]:
# Standard library imports
from typing import List, Optional, Dict, Any

# Third-party imports
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel, Field

class Embeddings(BaseModel):
    """Manages document embeddings for semantic search.
    
    Attributes:
        model_name: HuggingFace model identifier
        device: Computing device (cpu/cuda)
        encode_kwargs: Parameters for encoding
    """
    model_name: str = Field(default="sentence-transformers/all-mpnet-base-v2")
    device: str = Field(default="cpu")
    encode_kwargs: Dict[str, Any] = Field(default_factory=dict)
    
    def __init__(self, **kwargs):
        """Initialize the embedding model."""
        super().__init__(**kwargs)
        self._model = SentenceTransformer(
            self.model_name,
            device=self.device
        )
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute embeddings for multiple documents."""
        # Normalize newlines for consistency
        texts = [text.replace("\n", " ") for text in texts]
        
        # Generate embeddings
        embeddings = self._model.encode(
            texts,
            **self.encode_kwargs
        )
        
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Embed a single query string."""
        return self.embed_documents([text])[0]

In [3]:
#######################################
# Chat Template 
#######################################

import guidance
from guidance import models, gen, select, user, system, assistant
from guidance.chat import ChatTemplate

class InstructTemplate(ChatTemplate):
    """Formats chat messages for instruction-tuned models."""
    
    def get_role_start(self, role_name: str) -> str:
        """Get the prefix for a role's message."""
        prefixes = {
            "system": "System: ",
            "user": "User: ",
            "assistant": "Assistant: "
        }
        if role_name not in prefixes:
            raise ValueError(f"Unsupported role: {role_name}")
        return prefixes[role_name]

    def get_role_end(self, role_name: Optional[str] = None) -> str:
        """Get the suffix for a role's message."""
        return "\n"

In [4]:
# Standard library imports
import os
from typing import Optional, List

# Third-party imports
import chromadb
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from guidance import models
from rich import print
from rich.panel import Panel

class LoreManager:
    """Central manager for the lore system.
    
    Handles:
    - Vector store operations
    - Language model operations
    - Lore generation and refinement
    """
    
    def __init__(self, 
                 model_path: str,
                 embeddings_path: str,
                 db_path: str = "./lore.db"):
        """Initialize the lore management system.
        
        Args:
            model_path: Path to language model
            embeddings_path: Path to embedding model
            db_path: Path to database
        """
        self.db_path = db_path
        self.setup_models(model_path, embeddings_path)
        self.setup_database()
        
    def setup_models(self, model_path: str, embeddings_path: str):
        """Initialize language and embedding models."""
        # Setup embedding model
        self.embeddings = Embeddings(
            model_name=embeddings_path,
            device="cpu"  # Embeddings typically don't need GPU
        )
        
        # Setup language model with quantization
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )
        
        # Load model and tokenizer
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="cuda",  # Language model benefits from GPU
            quantization_config=quant_config,
            torch_dtype=torch.float16
        )
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        
        # Create guidance model
        self.model = models.Transformers(
            model=model,
            tokenizer=tokenizer,
            chat_template=InstructTemplate,
            device_map="cuda",
            echo=False
        )
        
    def setup_database(self):
        """Initialize the vector database."""
        self.client = chromadb.PersistentClient(path=self.db_path)
        self.collection = self.client.get_or_create_collection(
            name="lore_collection"
        )
        
    def add_lore(self, entry: LoreEntry):
        """Add a lore entry to the vector store."""
        self.collection.add(
            ids=[entry.name],
            documents=[entry.content],
            metadatas=[{"keywords": " ".join(entry.keywords)}]
        )
        
    def search_lore(self, query: str, n_results: int = 3) -> List[dict]:
        """Search for relevant lore entries."""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        return results['documents'][0] if results['documents'] else []
        
    def generate_lore(self, world: World, focus: str) -> LoreEntry:
        """Generate new lore entry."""
        # Get context from existing lore
        context = self.search_lore(focus)
        context = "\n- ".join(context) if context else "No existing lore."
        
        # Generate initial proposal
        with guidance.user():
            result = self.model + f"""Create lore for a {world.setting} world focused on: {focus}
            
            World Context:
            {world.world_description}
            
            Existing Lore:
            {context}
            
            Generate something that:
            1. Focuses on {focus}
            2. Fits the world theme
            3. Builds on existing lore
            4. Has rich details
            """
            
        # Generate structured response
        with guidance.assistant():
            result += "Name: " + guidance.gen(name="name", stop="\n")
            result += "\nContent: " + guidance.gen(name="content", stop="\n")
            result += "\nKeywords: " + guidance.gen(name="keywords", stop="\n")
            
        # Create and return lore entry
        return LoreEntry(
            name=result["name"],
            content=result["content"],
            keywords=[k.strip() for k in result["keywords"].split(",")]
        )
        
    def cleanup(self):
        """Clean up resources."""
        if hasattr(self.model, 'cleanup'):
            self.model.cleanup()

    def propose_lore(self, world: World, focus: str) -> LoreProposal:
        """Generate initial lore proposal with questions."""
        
        # Get context from existing lore
        context = self.search_lore(focus)
        context = "\n- ".join(context) if context else "No existing lore."

        with guidance.system():
            result = self.model + """You are a world-building assistant that proposes new lore entries and identifies key questions for refinement."""

        with guidance.user():
            result += f"""### Instruction
    Create an initial lore proposal and identify key questions to ensure consistency and completeness.

    ### Input
    World Setting: {world.setting.value}
    World Description: {world.world_description}
    Focus Topic: {focus}

    Existing Lore:
    {context}

    Requirements:
    1. Create a detailed initial proposal about {focus}
    2. Generate 3 specific questions to improve the proposal
    3. Explain your reasoning for the questions
    4. Ensure consistency with existing lore
    5. Match the {world.setting.value} theme

    Output Format:
    Initial Proposal: [A detailed first draft of the lore]
    Questions:
    1. [First specific question to improve/verify the lore]
    2. [Second specific question]
    3. [Third specific question]
    Reasoning: [Explain why these questions are important]

    ### Output
    """

        with guidance.assistant():
            result += "Initial Proposal: " + guidance.gen(
                name="proposal", 
                stop="\n\n",
                max_tokens=350,
                temperature=0.8
            ) + "\n\n"
            
            result += "Questions:\n"
            for i in range(3):
                result += f"{i+1}. " + guidance.gen(
                    name=f"question_{i}",
                    stop="\n",
                    max_tokens=100,
                    temperature=0.7
                ) + "\n"
                
            result += "\nReasoning: " + guidance.gen(
                name="reasoning",
                stop="\n\n",
                max_tokens=200,
                temperature=0.7
            )

        return LoreProposal(
            initial_proposal=result["proposal"],
            questions=[result[f"question_{i}"] for i in range(3)],
            reasoning=[result["reasoning"]]
        )

    def refine_proposal(self, proposal: LoreProposal, world: World) -> LoreEntry:
        """Refine proposal using search results."""
        refinements = []
        
        # Get refinements for each question
        for question in proposal.questions:
            context = self.search_lore(question)
            context = "\n- ".join(context) if context else "No existing lore."
            
            with guidance.system():
                result = self.model + """You are a world-building assistant that refines lore proposals based on existing knowledge."""
                
            with guidance.user():
                result += f"""### Instruction
    Analyze the question and relevant lore to provide a refinement for the proposal.

    ### Input
    Question: {question}
    Relevant Lore:
    {context}

    Requirements:
    1. Analyze how the existing lore impacts the proposal
    2. Suggest specific improvements or changes
    3. Maintain consistency with established facts
    4. Keep the {world.setting.value} theme
    5. Provide concrete details

    ### Output
    """
            
            with guidance.assistant():
                result += guidance.gen(
                    name="refinement",
                    stop="\n\n",
                    max_tokens=250,
                    temperature=0.7
                )
            
            refinements.append(result["refinement"])

        # Generate final lore entry
        with guidance.system():
            result = self.model + """You are a world-building assistant that creates polished final lore entries."""
            
        with guidance.user():
            result += f"""### Instruction
    Create a final lore entry that incorporates the original proposal and refinements.

    ### Input
    Original Proposal:
    {proposal.initial_proposal}

    Refinements:
    """ + "\n".join(f"- {r}" for r in refinements) + """

    Requirements:
    1. Create a cohesive entry that integrates all refinements
    2. Maintain the {world.setting.value} theme
    3. Ensure internal consistency
    4. Include rich, specific details
    5. Provide clear categorization keywords

    Output Format:
    Name: [A clear, descriptive title]
    Content: [The complete, refined lore text]
    Keywords: [3-5 relevant search terms]

    ### Output
    """
            
        with guidance.assistant():
            result += "Name: " + guidance.gen(name="name", stop="\n", max_tokens=100, temperature=0.7) + "\n"
            result += "Content: " + guidance.gen(name="content", stop="\n", max_tokens=500, temperature=0.8) + "\n"
            result += "Keywords: " + guidance.gen(name="keywords", stop="\n", max_tokens=50, temperature=0.7)
            
        return LoreEntry(
            name=result["name"],
            content=result["content"],
            keywords=[k.strip() for k in result["keywords"].split(",")]
        )
    
    def query_lore(self, query: str) -> str:
        """Query and analyze lore to answer questions.
        
        Args:
            query: The question to answer about the lore
            
        Returns:
            str: A detailed response based on available lore
        """
        # Get relevant context from existing lore
        context = self.search_lore(query)
        context = "\n- ".join(context) if context else "No existing lore."

        # Start system context
        with guidance.system():
            result = self.model + """You are a knowledgeable lore expert who answers questions based strictly on provided information."""

        # Provide instruction and input
        with guidance.user():
            result += f"""### Instruction
    Answer the query by analyzing and synthesizing information from the provided lore. Only use facts explicitly stated in the lore.

    ### Input
    Query: {query}
    Available Lore:
    {context}

    Requirements:
    1. Answer should be detailed and specific
    2. Only use information from the provided lore
    3. Clearly indicate if information is uncertain or missing
    4. Provide a single comprehensive paragraph

    ### Output
    """

        # Get structured response
        with guidance.assistant():
            result += guidance.gen(
                name="response",
                stop="\n\n",
                max_tokens=500,
                temperature=0.8
            )

        return result["response"]

In [5]:
# Standard library imports
import os
from typing import Optional

# Third-party imports
from rich import print
from rich.panel import Panel
from rich.markdown import Markdown
from rich.progress import Progress

MODEL_PATH = r'C:\Users\polym\.cache\huggingface\hub\models--microsoft--Phi-3.5-mini-instruct\snapshots\af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0'
EMBEDDINGS_PATH = r"C:\Users\polym\.cache\torch\sentence_transformers\BAAI_bge-large-en-v1.5"

def build_world(world_type: SettingType, core_concept: str):
    """Build a world with proposal and refinement cycles.
    
    Args:
        world_type: The genre/setting type
        core_concept: Core world concept to build around
    """
    
    # Initialize the lore manager
    lore_manager = LoreManager(
        model_path=MODEL_PATH,
        embeddings_path=EMBEDDINGS_PATH
    )
    
    try:
        # Create the world
        world = World(
            setting=world_type,
            world_description=core_concept
        )
        
        # Display world info
        print(Panel.fit(
            world.world_description + f"\n\nSetting: {world.setting}",
            title="🌍 World Description",
            border_style="cyan"
        ))
        
        # Define foundation aspects
        foundation_aspects = [
            "basic principles",
            "key powers",
            "major factions",
            "important locations",
            "historical events"
        ]
        
        # Generate lore for each aspect
        for aspect in foundation_aspects:
            print(Panel.fit(
                f"Generating foundational lore about {aspect}...",
                title="⚡ New Aspect",
                border_style="blue"
            ))
            
            # Step 1: Generate initial proposal
            proposal = lore_manager.propose_lore(world, aspect)
            
            # Display proposal and questions
            print(Panel.fit(
                Markdown(f"""
                **Initial Proposal:**
                {proposal.initial_proposal}
                
                **Questions to Explore:**
                """ + "\n".join(f"- {q}" for q in proposal.questions) + f"""
                
                **Reasoning:**
                {proposal.reasoning[0]}
                """),
                title="📝 Initial Proposal",
                border_style="yellow"
            ))
            
            # Step 2: Refine proposal
            print("\nRefining proposal based on existing lore...")
            final_lore = lore_manager.refine_proposal(proposal, world)
            
            # Step 3: Store refined lore
            lore_manager.add_lore(final_lore)
            
            # Display final lore
            print(Panel.fit(
                Markdown(f"""
                **Content:**
                {final_lore.content}
                
                **Keywords:** {', '.join(final_lore.keywords)}
                """),
                title=f"✨ {final_lore.name}",
                border_style="green"
            ))
        
        print("\nAnalyzing relationships between aspects...")
        relationship_queries = [
            "How do the basic principles affect the key powers?",
            "What role do major factions play in important locations?",
            "How have historical events shaped the current state?",
            "What conflicts exist between different factions?",
            "How do locations influence power dynamics?"
        ]
        qa = []

        # Get insights for each relationship query
        for query in relationship_queries:
            # Get analysis using LoreManager's query method
            analysis = lore_manager.query_lore(query)
            
            
            qa.append({"question": query, "answer":analysis})

        for x in qa:
            
            # Display the results
            print(Panel.fit(
                Markdown(f"""
                **Query:**
                {x["question"]}

                **Analysis:**
                {x["answer"]}
                """),
                title="🔄 Relationship Analysis",
                border_style="magenta"
            ))
            
    finally:
        # Clean up resources
        lore_manager.cleanup()
        print(Panel.fit(
            "World building completed!",
            border_style="green"
        ))

if __name__ == "__main__":
    print(Panel.fit(
        "Welcome to the World Builder!\n\nEnter a core concept for your world:",
        title="🎮 World Builder",
        border_style="blue"
    ))
    
    concept = input("> ")
    
    build_world(
        world_type=SettingType.horror,
        core_concept=concept
    )

╭────────── 🎮 World Builder ──────────╮
│ Welcome to the World Builder!        │
│                                      │
│ Enter a core concept for your world: │
╰──────────────────────────────────────╯

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

╭──────────────────────────────────────── 🌍 World Description ────────────────────────────────────────╮
│ A massive dark fantasy work with themes of cosmic horror, political intrigue, and economic scarcity. │
│                                                                                                      │
│ Setting: SettingType.horror                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────── ⚡ New Aspect ─────────────────────╮
│ Generating foundational lore about basic principles... │
╰────────────────────────────────────────────────────────╯

We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)


...

╭─────────────────────────────────────────── 🔄 Relationship Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│              **Query:**                                                                                         │
│              What role do major factions play in important locations?                                           │
│                                                                                                                 │
│              **Analysis:**                                                                                      │
│              In the cosmic hierarchy influenced by Occultite, The Shrouded Monoliths, The Shattered Echoes, an  │
│  The Cryptic Whispers play critical roles as epicenters of occult phenomena within three paramount locations:   │
│  The Shrouded Monoliths, The Shattered Echoes, and The Cryptic Whispers. The Shrouded Monoliths, The Shattered  │
│  Echoes, and The Cryptic Whispers are interconnected through the Nexus of Fates, a sacred network that defines  │
│  the triad's influence and power. These locations are crucibles of cosmic disturbansions, shaping reality's     │
│  fabric within the shadow-cloaked expanse governed by The Void. The Sundering, a monumental event embodying th  │
│  convergence of past, present, and future into chaos, occurs along these sacred pathways, merging the boundari  │
│  of existence and underscoring the triad's role in pivotal happenings. The Shrouded Monoliths, The Shattered    │
│  Echoes, and The Cryptic Whispers, therefore, are not merely points of interest but active participants in the  │
│  unfolding of cosmic events, each possessing unique esoteric endowments that contribute to the vast, intertwin  │
│  narrative of this universe.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔄 Relationship Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│              **Query:**                                                                                         │
│              How have historical events shaped the current state?                                               │
│                                                                                                                 │
│              **Analysis:**                                                                                      │
│              The Sundering, as a monumental cosmic event, has drastically reshaped the fabric of reality along  │
│  the Nexus of Fates, merging past, present, and future into a singular nexus of chaos. This event has not only  │
│  altered the physical structures of The Shrouded Monoliths, The Shattered Echoes, and The Cryptic Whispers—thr  │
│  formidable factions that wield LagRarens, the mystical force known as Occultite—but has also transformed the   │
│  cosmic hierarchy into a realm of perpetual reconstruction. The Sundering's impact is evident in the intertwin  │
│  existence of these factions, suggesting that the historical events, especially The Sundering, have been        │
│  instrumental in shaping their current state within the cosmos. Given the lore's depiction of reality as a      │
│  fragile construct under The Void's influence, it can be inferred that The Sundering has further destabilized   │
│  this balance, making the future uncertain for these entities. However, without explicit details on the direct  │
│  consequences of The Sundering on each faction's structure or power, the exact nature of this transformation    │
│  remains speculative.                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔄 Relationship Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│              **Query:**                                                                                         │
│              What conflicts exist between different factions?                                                   │
│                                                                                                                 │
│              **Analysis:**                                                                                      │
│              The lore suggests a complex and delicate balance among the factions of The Shrouded Monoliths, Th  │
│  Shattered Echoes, and The Cryptic Whispers, each influenced by Occultite, a mystical force that binds them     │
│  together. The Sundering, a cosmic cataclysm described as merging past, present, and future into chaos, furthe  │
│  complicates the inter-faction dynamics within this fragile cosmic construct. While the lore details the event  │
│  and locations associated with these factions, it does not explicitly state the nature of the conflicts betwee  │
│  them. Given the descriptions, it can be inferred that these factions likely engage in a struggle for dominanc  │
│  within the cosmic hierarchy, with their powers tied to the Nexus of Fates and Occultite. However, since the    │
│  specific conflicts are not detailed, the exact nature of their disputes remains uncertain within the provided  │
│  information.                                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔄 Relationship Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│              **Query:**                                                                                         │
│              How do locations influence power dynamics?                                                         │
│                                                                                                                 │
│              **Analysis:**                                                                                      │
│              In the lore provided, locations such as The Shrouded Monoliths, The Shattered Echoes, and The      │
│  Cryptic Whispers play a crucial role in influencing power dynamics within our universe, which is said to be    │
│  under the malevolent control of The Void. The power of Occultite, a form of mystical endowment only accessibl  │
│  to those whose spirits align with The Void's rhythms or who hail from sanctified lineages, is intimately tied  │
│  to these locations. It is implied that these sites are epicenters of cosmic disturbances and powerful occult   │
│  phenomena, suggesting their importance in the distribution and exercise of Occultite powers. The Shrouded      │
│  Monoliths, The Shattered Echoes, and The Cryptic Whispers form a triad of influence, each exerting esoteric    │
│  control over reality's fabric by enabling their bearers to perceive, mold, and transcend the mundane. The      │
│  specific ways in which these locations directly shape power dynamics are not detailed in the lore; however, i  │
│  is clear that they are essential in the cosmic hierarchy, likely influencing power dynamics through the wield  │
│  Occultite powers. The exact nature of this influence remains uncertain, but the lore underscores the           │
│  significance of these locations in the broader cosmic and occult landscape.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────╮
│ World building completed! │
╰───────────────────────────╯

In [ ]:
# Standard library imports
import os
from typing import Optional

# Third-party imports
from rich import print
from rich.panel import Panel


MODEL_PATH = r'C:\Users\polym\.cache\huggingface\hub\models--microsoft--Phi-3.5-mini-instruct\snapshots\af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0'
    
#MODEL_PATH = r"C:\Users\polym\.cache\huggingface\hub\models--tiiuae--Falcon3-7B-Instruct\snapshots\28519b87831eaf6dbe6962f889b82b5a25b5d940"
    
EMBEDDINGS_PATH = r"C:\Users\polym\.cache\torch\sentence_transformers\BAAI_bge-large-en-v1.5"


def build_world(world_type: SettingType, core_concept: str):
    """Build a world focused on a specific concept.
    
    Args:
        world_type: The genre/setting type
        core_concept: Core world concept to build around
    """
    
    # Initialize the lore manager with model paths
    lore_manager = LoreManager(
        model_path=MODEL_PATH,
        embeddings_path=EMBEDDINGS_PATH
    )
    
    try:
        # Create the world
        world = World(
            setting=world_type,
            world_description=core_concept
        )
        
        # Display world info
        print(Panel.fit(
            world.world_description + f"\n\nSetting: {world.setting}",
            title="World Description",
            border_style="cyan"
        ))
        
        # Generate foundation lore
        foundation_aspects = [
            "basic principles",
            "key powers",
            "major factions",
            "important locations",
            "historical events"
        ]
        
        # Generate lore for each aspect
        for aspect in foundation_aspects:
            print(f"\nGenerating lore for: {aspect}")
            
            # Generate and store lore
            lore = lore_manager.generate_lore(world, aspect)
            lore_manager.add_lore(lore)
            
            # Display generated lore
            print(Panel.fit(
                lore.content + f"\n\nKeywords: {', '.join(lore.keywords)}",
                title=lore.name,
                border_style="green"
            ))
            
    finally:
        # Clean up resources
        lore_manager.cleanup()

if __name__ == "__main__":
    # Get user input for world creation
    print("Enter a core concept for the world:")
    concept = input("> ")
    
    # Create a horror world with water theme
    build_world(
        world_type=SettingType.horror,
        core_concept=concept
    )

In [6]:
# Standard library imports
import os
from typing import Optional

# Third-party imports
from rich import print
from rich.panel import Panel


MODEL_PATH = r'C:\Users\polym\.cache\huggingface\hub\models--microsoft--Phi-3.5-mini-instruct\snapshots\af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0'
    
#MODEL_PATH = r"C:\Users\polym\.cache\huggingface\hub\models--tiiuae--Falcon3-7B-Instruct\snapshots\28519b87831eaf6dbe6962f889b82b5a25b5d940"
    
EMBEDDINGS_PATH = r"C:\Users\polym\.cache\torch\sentence_transformers\BAAI_bge-large-en-v1.5"


def build_world(world_type: SettingType, core_concept: str):
    """Build a world focused on a specific concept.
    
    Args:
        world_type: The genre/setting type
        core_concept: Core world concept to build around
    """
    
    # Initialize the lore manager with model paths
    lore_manager = LoreManager(
        model_path=MODEL_PATH,
        embeddings_path=EMBEDDINGS_PATH
    )
    
    try:
        # Create the world
        world = World(
            setting=world_type,
            world_description=core_concept
        )
        
        # Display world info
        print(Panel.fit(
            world.world_description + f"\n\nSetting: {world.setting}",
            title="World Description",
            border_style="cyan"
        ))
        
        # Generate foundation lore
        foundation_aspects = [
            "basic principles",
            "key powers",
            "major factions",
            "important locations",
            "historical events"
        ]
        
        # Generate lore for each aspect
        for aspect in foundation_aspects:
            print(f"\nGenerating lore for: {aspect}")
            
            # Generate and store lore
            lore = lore_manager.generate_lore(world, aspect)
            lore_manager.add_lore(lore)
            
            # Display generated lore
            print(Panel.fit(
                lore.content + f"\n\nKeywords: {', '.join(lore.keywords)}",
                title=lore.name,
                border_style="green"
            ))
            
    finally:
        # Clean up resources
        lore_manager.cleanup()

if __name__ == "__main__":
    # Get user input for world creation
    print("Enter a core concept for the world:")
    concept = input("> ")
    
    # Create a horror world with water theme
    build_world(
        world_type=SettingType.horror,
        core_concept=concept
    )

╭────────────────────────────────────────── The Echoes of Shadows Past ───────────────────────────────────────────╮
│ In the annals of WestWorld's history, The Echoes of Shadows Past stand as a testament to the park's enigmatic   │
│ origins. This chapter delves into the clandestine inception of The Shadowed Circuit, tracing its roots back to  │
│ a time when WestWorld was but a fledgling concept in the minds of its creators.                                 │
│                                                                                                                 │
│ Keywords: WestWorld, The Shadowed Circuit, Echolocators, historical events, Android Amusement Park              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯